In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

from data_load import get_encoded_data,get_clean_data

We need to create our feature vector of exact same dimension as our training set. To convert our user input into dummy variables, we should save a dict of the the dummy variables. Later we can populate our feature vector for prediction using this dict.


# Basic Naiyve Bayes Model with no Calibration

In [2]:
model_name = 'nb_model_basic'

In [3]:
data = get_encoded_data('quiz_data.csv','nb_model_basic',True)[0]

x_df = data.drop(axis=1,columns=["program"])
y_df = data["program"]

X = np.array(x_df) # convert dataframe into np array
y = np.array(y_df) # convert dataframe into np array

mnb = MultinomialNB()
model = mnb.fit(x_df, y_df) # fit the model using training data

cat = data.drop('program',axis=1)
index_dict = dict(zip(cat.columns,range(cat.shape[1])))

In [ ]:
# Model Accuracy, how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [10]:
# print(y)

[ 9  2 12 14  5  4  9 14 13  4 10  4  3  5 10 10  6 10 10 10  5 11  9  9
 10 11  4  9  8  7  6 12  7  7  7 10 10 13  7  9  9 13 11 14  5 10  2  7
  4  9  9  3  9  9 12 10  4  1  3 14 10 12  9  6  3  2  9 10  6 14  9  9
 10  7 14  9  0  7 10 10 12  5 14  3  7  2  6  4 11  3  9  9  3  4  9 13
 10  9  6  9 12  4  9  2 13  4  5 13 14  1  4  7  4  9 10 10  3 10 10  9
 10  9  9 10  8  6 11  4 11  7  5 12  1  4 10 14  8  7  9 14  7  2  2 13
  4 13  1  5 14  9  3 12  9  6  1  3 10 11  9  7  2 13 13  9  1 10  1  5
  9  2  2  9 14  9 14  4  9 11  5  4 14  4 13  9  5 12  8 14 10  3  3 10
  3  4  3 13 11  9 13  9  9 10  3 10 11 11  2  0  5 13 14 14  1  1  9  5
  1  9 13  0 12  9  2 14  4 13  3  0  9  1  1 10 10 11 12  5  3  1 12  9
  1  7  0  7  9  0  0  1  0 14 13 12 14  1  3  4  5  5  6  7  7  9  5  2
 10  1  2  3  5  5  5  9  1  3  9  5  5  7  7 14  3  3  7  4  5  3  9 14
  9  7  9  7  4 14  8  4  3  3 14  0  0  0 14  9  3  0  0  1  9  9  3 11
  6 14 14 14  0  5  9  9  4  7 11  7  2  5  3  2  7

In [11]:
new_vector = [None] * 25
new_vector[0] =  2   # {'problem_type': {'defined': 0, 'investigate': 2, 'discover': 1}}
new_vector[1] =  2   #{'creative': {'somewhat_creative': 2, 'creative': 0, 'not_creative': 1}}
new_vector[2] =  1   #{'outdoors': {'limited': 1, 'indoors': 0, 'outdoors': 2}}
new_vector[3] =  4   #{'career': {'moving_parts': 2, 'sensors': 6, 'programming': 4, 'optimizing': 3, 'resources': 5, 'buildings': 0, 'molecules': 1}}
new_vector[4] =  1   #{'group_work': {'occasionally': 1, 'yes': 2, 'no': 0}}
new_vector[5] =  4   #{'liked_courses': {'computer_science': 4, 'biology': 1, 'math': 8, 'physics': 9, 'chemistry': 3, 'business': 2, 'autoshop': 0, 'geography': 5, 'visual_arts': 10, 'language_arts': 7, 'history': 6}}
new_vector[6] =  6   #{'disliked_courses': {'history': 6, 'visual_arts': 10, 'geography': 5, 'chemistry': 3, 'physics': 9, 'biology': 1, 'language_arts': 7, 'business': 2, 'math': 8, 'computer_science': 4, 'autoshop': 0}}
new_vector[7] =  3   #{'programming': {'partial': 3, 'complete': 0, 'no': 2, 'interested': 1}}
new_vector[8] =  8   #{'join_clubs': {'robotics': 7, 'nan': 6, 'environment': 4, 'student_council': 8, 'business': 2, 'hacker_club': 5, 'autoshop': 1, 'art/design': 0, 'consulting': 3}}
new_vector[9] =  0   #{'not_clubs': {'art/design': 0, 'business': 2, 'student_council': 8, 'autoshop': 1, 'hacker_club': 5, 'environment': 4, 'robotics': 7, 'nan': 6, 'consulting': 3}}
new_vector[10] = 1   #{'liked_projects': {'mars_water_treatment': 1, 'prototyping_instrument': 3, 'robot': 4, 'olympic_village': 2, 'battery': 0, 'uber_pool': 6, 'supercomputer': 5}}
new_vector[11] = 1   #{'disliked_projects': {'prototyping_instrument': 3, 'mars_water_treatment': 1, 'robot': 4, 'uber_pool': 6, 'olympic_village': 2, 'battery': 0, 'supercomputer': 5}}
new_vector[12] = 0   #{'tv_shows': {'big_bang_theory': 0, 'breaking_bad': 1, 'myth_busters': 4, 'silicon_valley': 6, 'planet_earth': 5, 'greys_anatomy': 2, 'house_hunters': 3}}
new_vector[13] = 0   #{'alternate_degree': {'cs': 2, 'applied_science': 0, 'health': 8, 'env': 5, 'poli_sci': 12, 'econ': 4, 'math': 11, 'business': 1, 'design': 3, 'visual_arts': 14, 'psych': 13, 'geo': 7, 'fin': 6, 'marketing': 10, 'lit': 9}}
new_vector[14] = 1   #{'expensive_equipment': {'yes': 2, 'maybe': 0, 'no': 1}}
new_vector[15] = 2   #{'drawing': {'partial': 2, 'bad': 0, 'good': 1}}
new_vector[16] = 1   #{'essay': {'yes': 2, 'no': 0, 'partial': 1}}
new_vector[17] = 0   #{'architecture': {0: 0, 1: 1}}
new_vector[18] = 0   #{'automotive': {1: 1, 0: 0}}
new_vector[19] = 1   #{'business': {0: 0, 1: 1}}
new_vector[20] = 0   #{'construction': {1: 1, 0: 0}}
new_vector[21] = 0   #{'health': {0: 0, 1: 1}}
new_vector[22] = 1   #{'environment': {0: 0, 1: 1}}
new_vector[23] = 1   #{'manufacturing': {1: 1, 0: 0}}
new_vector[24] = 1   #{'technology': {1: 1, 0: 0}}

new_vector = np.array(new_vector).reshape(-1, 1)
print(new_vector)

# print("Loading model")
# pkl_file = open('nb_model.pkl', 'rb')
# print(pkl_file)
# nb_model = pickle.load(pkl_file)
# print(nb_model)
# prediction = nb_model.predict(new_vector)
# print(prediction)

model.predict(new_vector)

[[2]
 [2]
 [1]
 [4]
 [1]
 [4]
 [6]
 [3]
 [8]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [2]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]]


ValueError: shapes (25,1) and (25,15) not aligned: 1 (dim 1) != 25 (dim 0)

# Export Basic Naive Bayes Model

In [ ]:
nb_model = naive_bayes('quiz_data.csv')[0]
with open('exported_model_files/nb_model_basic.pkl', 'wb') as fid:
    pickle.dump(nb_model, fid,2)

index_dict = naive_bayes('golf_data.csv')[1]

with open('exported_model_files/nb_model_basic_cat', 'wb') as fid:
    pickle.dump(index_dict, fid,2)

# Encoded Dictionary
{'program': {'mech': 9, 'bmed': 2, 'swe': 12, 'tron': 14, 'cive': 5, 'chem': 4, 'syde': 13, 'msci': 10, 'ce': 3, 'elec': 6, 'nano': 11, 'geo': 8, 'env': 7, 'arch-e': 1, 'arch': 0}}

{'happy': {'Yes': 0}}

{'problem_type': {'defined': 0, 'investigate': 2, 'discover': 1}}

{'creative': {'somewhat_creative': 2, 'creative': 0, 'not_creative': 1}}

{'outdoors': {'limited': 1, 'indoors': 0, 'outdoors': 2}}

{'career': {'moving_parts': 2, 'sensors': 6, 'programming': 4, 'optimizing': 3, 'resources': 5, 'buildings': 0, 'molecules': 1}}

{'group_work': {'occasionally': 1, 'yes': 2, 'no': 0}}

{'liked_courses': {'computer_science': 4, 'biology': 1, 'math': 8, 'physics': 9, 'chemistry': 3, 'business': 2, 'autoshop': 0, 'geography': 5, 'visual_arts': 10, 'language_arts': 7, 'history': 6}}

{'disliked_courses': {'history': 6, 'visual_arts': 10, 'geography': 5, 'chemistry': 3, 'physics': 9, 'biology': 1, 'language_arts': 7, 'business': 2, 'math': 8, 'computer_science': 4, 'autoshop': 0}}

{'programming': {'partial': 3, 'complete': 0, 'no': 2, 'interested': 1}}

{'join_clubs': {'robotics': 7, 'nan': 6, 'environment': 4, 'student_council': 8, 'business': 2, 'hacker_club': 5, 'autoshop': 1, 'art/design': 0, 'consulting': 3}}

{'not_clubs': {'art/design': 0, 'business': 2, 'student_council': 8, 'autoshop': 1, 'hacker_club': 5, 'environment': 4, 'robotics': 7, 'nan': 6, 'consulting': 3}}

{'liked_projects': {'mars_water_treatment': 1, 'prototyping_instrument': 3, 'robot': 4, 'olympic_village': 2, 'battery': 0, 'uber_pool': 6, 'supercomputer': 5}}

{'disliked_projects': {'prototyping_instrument': 3, 'mars_water_treatment': 1, 'robot': 4, 'uber_pool': 6, 'olympic_village': 2, 'battery': 0, 'supercomputer': 5}}

{'tv_shows': {'big_bang_theory': 0, 'breaking_bad': 1, 'myth_busters': 4, 'silicon_valley': 6, 'planet_earth': 5, 'greys_anatomy': 2, 'house_hunters': 3}}

{'alternate_degree': {'cs': 2, 'applied_science': 0, 'health': 8, 'env': 5, 'poli_sci': 12, 'econ': 4, 'math': 11, 'business': 1, 'design': 3, 'visual_arts': 14, 'psych': 13, 'geo': 7, 'fin': 6, 'marketing': 10, 'lit': 9}}

{'expensive_equipment': {'yes': 2, 'maybe': 0, 'no': 1}}

{'drawing': {'partial': 2, 'bad': 0, 'good': 1}}

{'essay': {'yes': 2, 'no': 0, 'partial': 1}}

{'architecture': {0: 0, 1: 1}}

{'automotive': {1: 1, 0: 0}}

{'business': {0: 0, 1: 1}}

{'construction': {1: 1, 0: 0}}

{'health': {0: 0, 1: 1}}

{'environment': {0: 0, 1: 1}}

{'manufacturing': {1: 1, 0: 0}}

{'technology': {1: 1, 0: 0}}